In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import glob 
import aplpy
from tqdm.auto import tqdm
from astropy.utils.console import ProgressBar

import sys
sys.path.append('/Users/abarnes/GitHub/misc/data_cube_analysis/')

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in" 


import getdata
import moments
import rms
import masks

In [2]:
inputdir = '/Users/abarnes/Dropbox/work/Projects/IRAM-30m/cloudh_projects/cubes_processed'
inputfiles = glob.glob('%s/*.fits' %inputdir)
inputfiles = np.sort(inputfiles)
inputfiles.sort()

In [3]:
def makeplot(moms, inputfile, outputdir = './../figures/'):

    bbox = dict(facecolor='whitesmoke',  alpha=0.95, boxstyle='round')
    fig = plt.figure(figsize=(4, 4))
    ax = aplpy.FITSFigure(moms['max'], figure=fig)

    vmin, vmax = np.nanpercentile(moms['max'].data, (5, 99.9))
    ax.show_colorscale(vmin=vmin, vmax=vmax, cmap='inferno')

    ax.axis_labels.hide()
    ax.tick_labels.hide()

    ax.add_colorbar()
    ax.ticks.set_color('black')
    ax.colorbar.set_width(0.15)

    ax.add_label(0.05, 0.95, inputfile.split('/')[-1].split('.')[0].replace('CloudH_', ''),  ha='left', va='top', size=10, bbox = bbox, relative=True)

    ax.set_nan_color('lightgrey')

    ax_plot = fig.get_axes()[-2]
    ax_plot.grid(True, alpha=0.35, ls=':', color='white')

    ax.colorbar.set_axis_label_text('T (max) [K]')

    ax.colorbar.set_location('top')
    axcbar = fig.get_axes()[-1]
    axcbar.xaxis.set_ticks_position('top')

    ax_cbar = fig.get_axes()[-1]
    ax_cbar.yaxis.set_ticks_position('both')

    fig.tight_layout(h_pad=0, w_pad=0.3)

    outputfile = outputdir+inputfile.split('/')[-1].replace('.fits', '.png')
    # outputfile = inputfile.replace('.fits', '.png').replace('cubes_processed', 'figures')
    fig.savefig(outputfile, bbox_inches='tight', dpi=300)  
    plt.close('all')
    
    return()

In [4]:
mom_velocitydict = {'cloudH': np.array([35, 55])}
rms_velocitydict = {'cloudH': np.array([[-200, 30], [60, 200]])}

for inputfile in tqdm(inputfiles):

    if 'FTSOdp' in inputfile: 
        continue

    cloud = inputfile.split('/')[-1].split('-')[0]
    hdu = fits.open(inputfile)[0]
    
    hdu.header['BUNIT'] = 'K'
    hdu.header['CTYPE1'] = 'RA---TAN' 
    hdu.header['CTYPE2'] = 'DEC--TAN'
    hdu.header['RADESYS'] = 'ICRS'
    hdu.header['EQUINOX'] = 2000.0

    cube = getdata.get_cube(hdu)
    rms_map = rms.get_rms(cube, rms_velocitydict['cloudH'])

    mask = masks.get_expmask(cube, rms_map, hthresh=3, lthresh=2, beamarea=1, randomnoise=False)
    cube_masked = cube.with_mask(mask)
    velo = np.array([-200,150])

    moms = moments.get_momentmaps(cube_masked, rms_map, mom_velocitydict['cloudH'])
    makeplot(moms, inputfile)

    # outputfile = inputfile.replace(inputdir, './../mom_28').replace('.fits', '')
    # rms_map.writeto('%s_rms.fits' %outputfile, overwrite=True)
    # moms['mom0'].writeto('%s_mom.fits' %outputfile, overwrite=True)
    # moms['max'].writeto('%s_max.fits' %outputfile, overwrite=True)
    # moms['mom0s2n'].writeto('%s_mom0s2n.fits' %outputfile, overwrite=True)

  0%|          | 0/54 [00:00<?, ?it/s]